# Installing and importing modules

In [1]:
!pip install elasticsearch
!pip install jproperties

     |████████████████████████████████| 348kB 7.8MB/s 


In [44]:
from elasticsearch import Elasticsearch
from jproperties import Properties
import pickle
from elasticsearch import helpers

from tqdm import tqdm
from glob import glob
import re
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading respective pickle files

In [5]:
config = Properties()
with open('/content/drive/MyDrive/config/elastic_search_config.prop','rb') as f:
  config.load(f)

In [ ]:
with open('/content/drive/MyDrive/Pickle files/Product_level_dataframe','rb') as f:
  prod_df = pickle.load(f)

In [ ]:
prod_df.drop(index=142964,axis=0,inplace=True)

In [ ]:
#prod_df.drop(index=142967,axis=0

KeyError: ignored

In [22]:
with open('/content/drive/MyDrive/Pickle files/final_dataframe','rb') as f:
  image_df = pickle.load(f)

In [ ]:
image_df.reset_index(drop=False,inplace=True)

In [ ]:
image_df

,index,gender,category,sub-category,product-number,product-name,path
0,0,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
1,3,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
2,4,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
3,6,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
4,8,women,dresses,casual_and_day_dresses,62629380,multicolor henley t-shirt dress,women/dresses/casual_and_day_dresses/62629380/...
...,...,...,...,...,...,...,...
91460,201831,women,tops,t-shirts,91437517,classic tee black,women/tops/t-shirts/91437517/91437517_2.jpeg
91461,201832,women,tops,t-shirts,91437722,black red heart t-shirt,women/tops/t-shirts/91437722/91437722_0.jpeg
91462,201833,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_0.jpeg
91463,201834,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_1.jpeg


In [ ]:
image_df.drop(index=142964,axis=0,inplace=True)

In [36]:
image_paths = glob('/content/drive/MyDrive/npz files /*.npz')

In [37]:
len(image_paths)

91465

In [38]:
def sort_glob(file):
  file_path_dic ={}
  for f in file:
    file_path_dic[int(re.findall('[0-9]+',f)[0])] = f

  file_paths = []
  sorted_file_path = sorted(file_path_dic.items())

  for tup in sorted_file_path:
    file_paths.append(tup[1])
  
  return file_paths

In [39]:
image_paths = sort_glob(image_paths)
image_paths[:20]

['/content/drive/MyDrive/npz files /0.npz',
 '/content/drive/MyDrive/npz files /3.npz',
 '/content/drive/MyDrive/npz files /4.npz',
 '/content/drive/MyDrive/npz files /6.npz',
 '/content/drive/MyDrive/npz files /8.npz',
 '/content/drive/MyDrive/npz files /10.npz',
 '/content/drive/MyDrive/npz files /11.npz',
 '/content/drive/MyDrive/npz files /13.npz',
 '/content/drive/MyDrive/npz files /14.npz',
 '/content/drive/MyDrive/npz files /16.npz',
 '/content/drive/MyDrive/npz files /20.npz',
 '/content/drive/MyDrive/npz files /22.npz',
 '/content/drive/MyDrive/npz files /25.npz',
 '/content/drive/MyDrive/npz files /28.npz',
 '/content/drive/MyDrive/npz files /30.npz',
 '/content/drive/MyDrive/npz files /31.npz',
 '/content/drive/MyDrive/npz files /32.npz',
 '/content/drive/MyDrive/npz files /35.npz',
 '/content/drive/MyDrive/npz files /37.npz',
 '/content/drive/MyDrive/npz files /39.npz']

In [ ]:
import numpy as np
image_vector = {}
for x in tqdm(image_paths):
  vector = np.load(x)['arr_0']
  image_vector[int(re.findall('[0-9]+',x)[0])] = vector

  1%|          | 1020/91465 [04:01<6:07:57,  4.10it/s]

KeyboardInterrupt: ignored

In [ ]:
image_paths[0]

'/content/drive/MyDrive/npz files /0.npz'

# Elastic Search

In [6]:
es = Elasticsearch([{'host':config.get('endpoint').data,'port':config.get('port').data}])

In [7]:
es.ping()

True

In [ ]:
prod_df_json = prod_df.to_dict('records')

In [23]:
image_df_json = image_df.to_dict('records')

In [ ]:
es.indices.create(index='product-level',ignore=400)

{'acknowledged': True, 'index': 'product-level', 'shards_acknowledged': True}

In [ ]:
es.indices.delete(index='product-level',ignore=[400])

In [35]:
es.indices.create(index='image-level-1',ignore=400)

{'acknowledged': True, 'index': 'image-level-1', 'shards_acknowledged': True}

In [ ]:
#es.indices.delete(index='image-level',ignore=400)

{'acknowledged': True}

In [ ]:
prod_df_json[0]

{'category': 'dresses',
 'gender': 'women',
 'path': 'women/dresses/casual_and_day_dresses/56037632/56037632_0.jpeg',
 'product-name': 'black knit midi dress',
 'product-number': '56037632',
 'sub-category': 'casual_and_day_dresses'}

In [ ]:
image_df_json[0]

{'category': 'dresses',
 'gender': 'women',
 'index': 0,
 'path': 'women/dresses/casual_and_day_dresses/56037632/56037632_0.jpeg',
 'product-name': 'black knit midi dress',
 'product-number': '56037632',
 'sub-category': 'casual_and_day_dresses'}

# uploading data into ES

In [ ]:
def generator_prod(df):
  for i,x in enumerate(df):
    yield{
        '_index':'product-level',
        '_type':'_doc',
        '_id': i+1,
        '_source':{
            "product-number":x.get("product-number",""),
            "product-name":x.get("product-name",""),
            "category":x.get("category",""),
            "sub-category":x.get("sub-category",""),
            "image-path":x.get("path","")
        }
    }
  raise StopIteration

In [ ]:
try:
  res1 = helpers.bulk(es,generator_prod(prod_df_json))
  print('working')
except Exception as e:
  pass

In [41]:
def generator_image(df):
  for i,x in enumerate(df):
    image_vector = np.load(image_paths[i])["arr_0"]
    yield{
        '_index':'image-level-1',
        '_type':'_doc',
        '_id': i+1,
        '_source':{
            "product-number":x.get("product-number",""),
            "product-name":x.get("product-name",""),
            "category":x.get("category",""),
            "sub-category":x.get("sub-category",""),
            "image-path":x.get("path",""),
            "image-vector": image_vector
                    }
    }
  raise StopIteration

In [46]:
try:
  res2 = helpers.bulk(es,generator_image(image_df_json))
  print('working')
except Exception as e:
  pass
#res2 = helpers.bulk(es,generator_image(image_df_json))  

/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)
